# <center>Сентимент-анализ отзывов на товары

[Соревнование](https://www.kaggle.com/c/product-reviews-sentiment-analysis) на Kaggle

## <center> Постановка задачи и предисловие

К вашей компании пришел заказчик, которому нужно решение задачи анализа тональности отзывов на товары. Заказчик хочет, чтобы вы оценили возможное качество работы такого алгоритма на небольшой тестовой выборке. При этом больше никаких данных вам не предоставляется.

Вам предстоит посмотреть на предоставленные заказчиком отзывы, собрать похожие отзывы в качестве обучающей выборки, и поэкспериментировать с постановкой задачи (разметкой вашей выборки на позитивные и негативные примеры) так, чтобы результат на примерах заказчика был по возможности получше.

Обратите внимание, что заказчик предоставил всего 100 примеров в качестве тестовой выборки - ситуация, когда размеченных данных почти нет - вообще очень частая в индустриальном анализе данных. Конечно, эти отзывы можно было бы идеально разметить вручную и получить максимальное качество, но вы сами не заинтересованы в таком подходе, т.к. потом придется и на всех новых примерах демонстрировать заказчику идеальную работу, что, конечно, вряд ли будет по силам алгоритму.

In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
from functools import reduce

In [2]:
with open('data/test.csv', 'rb') as f:
    data = f.read()
test = [rev.text for rev in bs4.BeautifulSoup(data, 'lxml').find_all('review')]

In [3]:
len(test), test

(100,
 ['Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.\nНесколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.\n',
  'ценанадежность-неубиваемостьдолго держит батарею 4 дня стабильно как телефон, 3-4 как плеер если \rпостоянно долбиться в уши и звонить по паре часо на дню, игры и, конечно,  смс , в месяц около 200 шт набирается.\r Максимальное время работы 5 дней в щадящем режиме.2 simqwerty рулит -после нее набор смс на обычных сенсо

Переведем в ДатаФрейм

In [4]:
test_df = pd.DataFrame({'review':test})

In [5]:
test_df.head()

,review
0,"Ужасно слабый аккумулятор, это основной минус ..."
1,ценанадежность-неубиваемостьдолго держит батар...
2,"подробнее в комментариях\nК сожалению, факт по..."
3,я любительница громкой музыки. Тише телефона у...
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех..."


Видим, что это отзывы на **мобильные телефоны**.

Отзывы с разметкой для трейна будем парсить отсюда:    
https://torg.mail.ru/review/goods/mobilephones/


## <center>Part 1. Parsing debug (справочно, можно пропустить и перейти к Part 2)

Анализ кода:    
- оценка отзыва - `<span class="review-item__rating-counter">4</span>`

In [8]:
url = 'https://torg.mail.ru/review/goods/mobilephones/'
req = requests.get(url)
print(req)

<Response [200]>


In [12]:
text = req.text

In [11]:
parser = bs4.BeautifulSoup(text, 'lxml')
print(parser.prettify())

<!DOCTYPE html>
<html class="touch_no" lang="ru-RU">
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
  <meta content="IE=edge" http-equiv="x-ua-compatible"/>
  <!--DNS-prefetch-->
  <link href="//torg20.imgsmail.ru" rel="dns-prefetch"/>
  <link href="//img.imgsmail.ru" rel="dns-prefetch"/>
  <link href="//r.mradx.net" rel="dns-prefetch"/>
  <link href="//ad.mail.ru" rel="dns-prefetch"/>
  <!--end DNS-prefetch-->
  <meta content="origin-when-crossorigin" name="referrer"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="slrdPz1l1h4LZC1No1cswmtVRobaNw3EQdff4WX9Nk0=" name="verify-v1"/>
  <meta content="NNnTemZMQ4EAngQQJZBgUB/+5fYIXmou2Vz7ZNdKTb4=" name="verify-v1"/>
  <meta content="4074a26f36d025a32ba07c4e" name="verify-reformal"/>
  <meta content="2ff119a7ff6a118e" name="mailru-verification"/>
  <meta content="jmR9akgBiGuaSCCXTJEmSQ0i8mDJULwq8-3b3wToFbg" name="google-site

In [33]:
x = parser.find('div', attrs={'class':'review-item'})
# titles = [title.text for title in x.findAll('li')]
# titles
print(x.prettify())

<div class="review-item js-review_item js-ustat_block js-ustat_block_review_0 js-ustat_rid_30590275535691556">
 <div class="review-item__header">
  <div class="card__responses__good_information card__responses__good_information__type2 clearfix">
   <img alt="" height="70" src="https://torg20.imgsmail.ru/70x70/model/e9be5a46c05b9288844ef4610f2374b6?src=14547343%2F1.jpg&amp;version=&amp;hash=55f03e20aff50c18c280c8ea2b9129e7" title="" width="70"/>
   <div class="card__responses__good_information__name">
    <a href="https://torg.mail.ru/mobilephones/highscreen-power-ice-evo-id14547343/">
     Смартфон Highscreen Power Ice Evo
    </a>
   </div>
   <div class="card__responses__good_information__price">
    от
    <b>
     6 490
    </b>
    до
    <b>
     7 990
    </b>
    руб.
    <!--
-->
    <a href="https://torg.mail.ru/mobilephones/highscreen-power-ice-evo-id14547343/">
     3 предложения
    </a>
   </div>
   <div class="card__responses__good_information__rating">
    <div class="v

In [34]:
len(x)

5

In [46]:
print(list(x)[3].prettify())

<div class="review-item__body">
 <div class="review-item__info">
  <div class="review-item__info-column">
   <div class="review-item__publication-info">
    <span class="review-item__author">
     <span class="social-nets-icon social-nets-icon_type_mymailru">
     </span>
     Артем Попыкин
    </span>
    написал
    <a class="review-item__publication-date" href="https://torg.mail.ru/review/goods/mobilephones/highscreen-power-ice-evo-id14547343/?review_id=3458053#review-list" rel="nofollow">
     18 февраля 2018
    </a>
    отзыв о сотовом телефоне Power Ice Evo
   </div>
   <div class="review-item__rating">
    <div class="voter voter_size_medium voter_color_orange">
     <div class="voter__wrap">
      <div class="voter__item voter__item_type_full">
      </div>
     </div>
     <div class="voter__wrap">
      <div class="voter__item">
      </div>
     </div>
     <div class="voter__wrap">
      <div class="voter__item">
      </div>
     </div>
     <div class="voter__wrap">
    

In [52]:
[t.text for t in x.find_all('span', attrs={'class':"review-item__rating-counter"})]

['1']

In [55]:
int(x.find('span', attrs={'class':"review-item__rating-counter"}).text)

1

In [81]:
x.find('p', attrs={'class':"review-item__paragraph"}).text.replace('\n', '').replace('\t', '')

'Китайское фуфло'

In [59]:
[t.text for t in x.find_all('p', attrs={'class':"review-item__paragraph"})]

['\nКитайское фуфло\t\t\t\t\t\t\t\t\t\t\t\t',
 '\nТолько батарея.\t\t\t\t\t\t\t\t\t\t\t\t',
 '\nТормозит страшно. Самостоятельно обновляется при включении Wi-Fi, несмотря на то что функция отключена как таковая. Стекло, которое поставляется вместе с телефоном слабо реагирует на нажатие, чувствительность низкая. После обновления (самостоятельного) с Андроид 6 до 7, вообще практически перестал... Читать полностью ']

In [77]:
print(x.find('a', attrs={'class':'more'}).prettify())

<a class="more" data-stat-add='{"action":"read-more"}' full-text="Тормозит страшно. Самостоятельно обновляется при включении Wi-Fi, несмотря на то что функция отключена как таковая. Стекло, которое поставляется вместе с телефоном слабо реагирует на нажатие, чувствительность низкая. После обновления (самостоятельного) с Андроид 6 до 7, вообще практически перестал реагировать на нажатие, а если и среагирует (о чудо), то тут же как бы сам нажимается опять и все надо начинать сначала. Сервис центр нарушений почему то не выявил. Короче валяется в  шкафу, в коробке. Бесполезно потраченные деньги." href="#" lesstext="Скрыть" moretext="Читать полностью">
 Читать полностью
</a>



In [104]:
x.find('a', attrs={'class':'more'}).attrs['full-text']

'Тормозит страшно. Самостоятельно обновляется при включении Wi-Fi, несмотря на то что функция отключена как таковая. Стекло, которое поставляется вместе с телефоном слабо реагирует на нажатие, чувствительность низкая. После обновления (самостоятельного) с Андроид 6 до 7, вообще практически перестал реагировать на нажатие, а если и среагирует (о чудо), то тут же как бы сам нажимается опять и все надо начинать сначала. Сервис центр нарушений почему то не выявил. Короче валяется в  шкафу, в коробке. Бесполезно потраченные деньги.'

In [99]:
print(parser.find_all('div', attrs={'class':'review-item'})[-1].prettify())

<div class="review-item js-review_item js-ustat_block js-ustat_block_review_19 js-ustat_rid_30590275535691556">
 <div class="review-item__header">
  <div class="card__responses__good_information card__responses__good_information__type2 clearfix">
   <img alt="" height="70" src="https://torg20.imgsmail.ru/70x70/model/6de70946c05b2659cdd7a3cad5619070?src=14554261%2F1.jpg&amp;version=&amp;hash=85a12ceac170aef7858641bb97828a15" title="" width="70"/>
   <div class="card__responses__good_information__name">
    <a href="https://torg.mail.ru/mobilephones/bq-mobile-bq-5035-velvet-id14554261/">
     Смартфон BQ Mobile BQ-5035 Velvet
    </a>
   </div>
   <div class="card__responses__good_information__price">
    <b>
     3 890
    </b>
    руб.
    <!--
-->
    <a href="https://torg.mail.ru/mobilephones/bq-mobile-bq-5035-velvet-id14554261/">
     2 предложения
    </a>
   </div>
   <div class="card__responses__good_information__rating">
    <div class="voter voter_size_small voter_color_orange"

In [102]:
print(parser.find_all('div', attrs={'class':'review-item'})[1].prettify())

<div class="review-item js-review_item js-ustat_block js-ustat_block_review_1 js-ustat_rid_30590275535691556">
 <div class="review-item__header">
  <div class="card__responses__good_information card__responses__good_information__type2 clearfix">
   <img alt="" height="70" src="https://torg20.imgsmail.ru/70x70/model/c6563141ef71a1c89c2117facd36742f?src=14536419%2F1.jpg&amp;version=&amp;hash=c38b648479958be0afb50f03c0947991" title="" width="70"/>
   <div class="card__responses__good_information__name">
    <a href="https://torg.mail.ru/mobilephones/oukitel-u22-id14536419/">
     Смартфон Oukitel U22
    </a>
   </div>
   <div class="card__responses__good_information__price">
    нет в продаже
    <!--
-->
   </div>
   <div class="card__responses__good_information__rating">
    <div class="voter voter_size_small voter_color_orange">
     <div class="voter__wrap">
      <div class="voter__item voter__item_type_full">
      </div>
     </div>
     <div class="voter__wrap">
      <div class=

Итак, парсим с **одной страницы** пары **(отзыв, оценка)**.   
Причем - организуем так, что если отзыв целиком в поле **.text** не вместился, то парсится **отзыв целиком** из поля **.attrs['full-text']**

In [106]:
[(item.find('a', attrs={'class':'more'}).attrs['full-text'] if item.find('a', attrs={'class':'more'}) else 
    item.find('p', attrs={'class':"review-item__paragraph"}).text.replace('\n', '').replace('\t', ''), 
  float(item.find('span', attrs={'class':"review-item__rating-counter"}).text.replace(',', '.')))
 for item in  parser.find_all('div', attrs={'class':'review-item'})]

[('Тормозит страшно. Самостоятельно обновляется при включении Wi-Fi, несмотря на то что функция отключена как таковая. Стекло, которое поставляется вместе с телефоном слабо реагирует на нажатие, чувствительность низкая. После обновления (самостоятельного) с Андроид 6 до 7, вообще практически перестал реагировать на нажатие, а если и среагирует (о чудо), то тут же как бы сам нажимается опять и все надо начинать сначала. Сервис центр нарушений почему то не выявил. Короче валяется в  шкафу, в коробке. Бесполезно потраченные деньги.',
  1.0),
 ('Очень хороший телефон, пользуюсь третий месяц.', 5.0),
 ('Айфон спорный аппарат. Начну с недостатков. Аккумулятор в нем ерунда. Не то как он его держит- а 1400- сам объем. Ну, понятно что аппарат 2011 года (что по нему не скажешь). При 10% заряда и ниже- просто отключается. Андроиды могут работать на 1 % заряда почти сутки- без использования.И до часа (+-)-в- использовании.Связь. Не знаю как у кого. Было проверено- Билайн- Кселл- ловит плохо. Это б

Соберем все вместе:

In [111]:
%%time
url = 'https://torg.mail.ru/review/goods/mobilephones/'
text = requests.get(url).text
parser = bs4.BeautifulSoup(text, 'lxml')
reviews_page1 = [(item.find('a', attrs={'class':'more'}).attrs['full-text'] if item.find('a', attrs={'class':'more'}) else 
    item.find('p', attrs={'class':"review-item__paragraph"}).text.replace('\n', '').replace('\t', ''), 
  float(item.find('span', attrs={'class':"review-item__rating-counter"}).text.replace(',', '.')))
 for item in  parser.find_all('div', attrs={'class':'review-item'})]

Wall time: 1.53 s


In [112]:
reviews_page1

[('Тормозит страшно. Самостоятельно обновляется при включении Wi-Fi, несмотря на то что функция отключена как таковая. Стекло, которое поставляется вместе с телефоном слабо реагирует на нажатие, чувствительность низкая. После обновления (самостоятельного) с Андроид 6 до 7, вообще практически перестал реагировать на нажатие, а если и среагирует (о чудо), то тут же как бы сам нажимается опять и все надо начинать сначала. Сервис центр нарушений почему то не выявил. Короче валяется в  шкафу, в коробке. Бесполезно потраченные деньги.',
  1.0),
 ('Очень хороший телефон, пользуюсь третий месяц.', 5.0),
 ('Айфон спорный аппарат. Начну с недостатков. Аккумулятор в нем ерунда. Не то как он его держит- а 1400- сам объем. Ну, понятно что аппарат 2011 года (что по нему не скажешь). При 10% заряда и ниже- просто отключается. Андроиды могут работать на 1 % заряда почти сутки- без использования.И до часа (+-)-в- использовании.Связь. Не знаю как у кого. Было проверено- Билайн- Кселл- ловит плохо. Это б

<h2 id="Part2"><center> Part 2. Parsing and preprocessing</center></h2>

Обозначения для трейна и теста:
- **train_df** - спарсенные 200 страниц (первоначальный датасет)
- **train_df900** - спарсенные 900 страниц (более весомый поздний датасет)
- **test_df['review_pre']** - обработанный **test** по первоначальному сценарию, который был для **train_df**
- **test_df['review_pre_900']** - обработанный **test** по более позднему сценарию, который был уже для **train_df900**

С учетом результата Части 1, спарсим страницы (по 20 отзывов на странице):

#### Сделаем функцию для парсинга:
(с обработкой исключений)

In [34]:
def parse_page(url):
    request = requests.get(url)
    # если возникает ошибка (страница не найдена (404) или запрос прерван (443) или еще что-то)
    # то генерируется исключение и печатается сообщение об этом, но код не прерывается, а выполняется дальше
    # в "тексте" при этом генерируется пустота (?) и записывается она как NaN (его потом надо будет отфильтровать)
    try:
        request.raise_for_status()
#     except Exception as e:
    except requests.exceptions.HTTPError as e:
        print('ERROR: %s' % e)

    text = request.text
    parser = bs4.BeautifulSoup(text, 'lxml')
    # парсим страницу
    page_reviews = [(item.find('a', attrs={'class':'more'}).attrs['full-text'] 
                     if item.find('a', attrs={'class':'more'}) else 
                     item.find('p', attrs={'class':"review-item__paragraph"}).text, 
                     float(item.find('span', attrs={'class':"review-item__rating-counter"}).text.replace(',', '.')))
                    for item in  parser.find_all('div', attrs={'class':'review-item'})]
    return page_reviews

тестируем

In [35]:
%%time
# список страниц для парсинга
n_start, n_end = 1, 201
url_list = ['http://httpbin.org/status/443', 'https://torg.mail.ru/review/goods/mobilephones/?page=2']
# парсим
map_results = map(parse_page, url_list)
reduce_results = reduce(lambda x,y: x + y, map_results)

ERROR: 443 Client Error: UNKNOWN for url: http://httpbin.org/status/443
Wall time: 2.55 s


In [389]:
%%time
# список страниц для парсинга
n_start, n_end = 1, 201
url_list = ['https://torg.mail.ru/review/goods/mobilephones/?page=' + str(num) for num in range(n_start, n_end)]
# парсим
map_results = map(parse_page, url_list)
reduce_results = reduce(lambda x,y: x + y, map_results)

Wall time: 7min 13s


In [392]:
len(reduce_results), reduce_results[:10]

(4000,
 [('Ужасная камера, отвратительная просто. Звук громкости, тихий очень. Подсветки на клавишах нет. Индикатора пропущенных вызовов нет. Не стоит таких денег, как на витрине. Хуже китайцев. А, но да за марку самсунг денег же содрать нужно. Фуфло полное. Кто думает его покупать. Сначала хорошенько все посмотрите и подумайте. Удачи!!!',
   2.0),
  ('Тормозит страшно. Самостоятельно обновляется при включении Wi-Fi, несмотря на то что функция отключена как таковая. Стекло, которое поставляется вместе с телефоном слабо реагирует на нажатие, чувствительность низкая. После обновления (самостоятельного) с Андроид 6 до 7, вообще практически перестал реагировать на нажатие, а если и среагирует (о чудо), то тут же как бы сам нажимается опять и все надо начинать сначала. Сервис центр нарушений почему то не выявил. Короче валяется в  шкафу, в коробке. Бесполезно потраченные деньги.',
   1.0),
  ('\nОчень хороший телефон, пользуюсь третий месяц.\t\t\t\t\t\t\t\t\t\t\t\t',
   5.0),
  ('Айфон спор

Мы получили список пар (отзыв, оценка).    
Сформируем из этого **обучающий** датафрейм.    
(как выше уже делали **тестовый**)

In [393]:
train_df = pd.DataFrame({'review':list(map(lambda x: x[0], reduce_results)), 
                         'rating':list(map(lambda x: x[1], reduce_results))})

In [394]:
train_df.head()

,rating,review
0,2.0,"Ужасная камера, отвратительная просто. Звук гр..."
1,1.0,Тормозит страшно. Самостоятельно обновляется п...
2,5.0,"\nОчень хороший телефон, пользуюсь третий меся..."
3,5.0,Айфон спорный аппарат. Начну с недостатков. Ак...
4,4.0,"\nПробыл 4 года,батарея стала держать максимум..."


In [398]:
train_df.shape

(4000, 2)

## Другой вариант парсинга - с другим (построчным в онлайне) способом сохранения

In [314]:
import csv
from tqdm import tqdm_notebook

def parse_torgmail_to_csv(url_list, path_and_filename='data/tmp.csv'):
    with open(path_and_filename, 'w+') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['review', 'rating']) # пишем заголовки колонок
        
        for url in tqdm_notebook(url_list, total=len(url_list)):
#             request = requests.get(url)
            
            # если возникает ошибка (страница не найдена (404) или запрос прерван (443) или еще что-то)
            # то генерируется исключение и печатается сообщение об этом, но код не прерывается, а выполняется дальше
            # в "тексте" при этом генерируется пустота (?) и записывается она как NaN (его потом надо будет отфильтровать)
            try:                              # обрабатываем возможные исключения
                request = requests.get(url)
                request.raise_for_status()
            except requests.exceptions.HTTPError as err:
#                 print('Oops. HTTP Error occured. Response is: {content} for url {url}'.format(content=err.response, url=url))
#                 print('Response is: {content}'.format(content=err.response.content))
                print('Oops. HTTP Error occured: %s' % err)
            except requests.exceptions.ConnectionError as err1:
#                 print('ERROR: %s' % err1.args[0])
                print('Oops. Connection Error occured. Response is: {content} for url {url}'.format(content=err1.response, 
                                                                                                    url=url))
            except requests.exceptions.RequestException as err2:
#                 print('ERROR: %s' % err2.args[0])
                print('Oops. RequestException Error occured. Response is: {content} for url {url}'.format(content=err2.response, 
                                                                                                    url=url))
            else:
                text = request.text
                parser = bs4.BeautifulSoup(text, 'lxml')    

                for item in  parser.find_all('div', attrs={'class':'review-item'}):


                    review = item.find('a', attrs={'class':'more'}).attrs['full-text'] \
                                    if item.find('a', attrs={'class':'more'}) \
                                    else item.find('p', attrs={'class':"review-item__paragraph"}).text
                    rating = float(item.find('span', attrs={'class':"review-item__rating-counter"}).text.replace(',', '.'))

                    try:
                        csvwriter.writerow([review, rating])
                    except Exception as err3:
                        print('Oops. Exception Error occured for url {url}. Response is: {content}'.format(url=url, 
                                                                                                           content=err3))    
   
    pass

тестируем

In [315]:
%%time
# список страниц для парсинга
# n_start, n_end = 1, 201
url_list = ['http://httpbin.org/status/443', 'https://torg.mail.ru/review/goods/mobilephones/?page=2', 
            'https://torg.mail.ru/review/goods/mobilephones/?page=3']
# парсим
parse_torgmail_to_csv(url_list, path_and_filename='data/tmp.csv')

A Jupyter Widget

Oops. Connection Error occured. Response is: None for url http://httpbin.org/status/443

Wall time: 11.4 s


In [316]:
tmp_df_by_csv = pd.read_csv('data/tmp.csv', index_col=None, header=0, encoding='cp1251')

print(tmp_df_by_csv.shape)
tmp_df_by_csv.head()

(40, 2)


,review,rating
0,"Купил спонтанно , зашёл в диксис и купил. Загр...",5.0
1,"\r\nВсегда был предан самсунгу, отдал за него ...",3.0
2,\r\nсенсор здох через 5 лет после покупки . ес...,4.5
3,Телефон отличный! Мощная батарея! По началу за...,5.0
4,\r\n в китае заказывал: Xiaomi Redmi Note 4X 1...,5.0


Парсим большой блок:

In [317]:
%%time
# список страниц для парсинга
n_start, n_end = 1, 901
url_list = ['https://torg.mail.ru/review/goods/mobilephones/?page=' + str(num) for num in range(n_start, n_end)]
# парсим
parse_torgmail_to_csv(url_list, path_and_filename='data/train_df900.csv')

A Jupyter Widget

Oops. Connection Error occured. Response is: None for url https://torg.mail.ru/review/goods/mobilephones/?page=16
Oops. Connection Error occured. Response is: None for url https://torg.mail.ru/review/goods/mobilephones/?page=33
Oops. Exception Error occured for url https://torg.mail.ru/review/goods/mobilephones/?page=46. Response is: 'charmap' codec can't encode character '\ufeff' in position 215: character maps to <undefined>
Oops. Connection Error occured. Response is: None for url https://torg.mail.ru/review/goods/mobilephones/?page=51
Oops. Connection Error occured. Response is: None for url https://torg.mail.ru/review/goods/mobilephones/?page=69
Oops. Connection Error occured. Response is: None for url https://torg.mail.ru/review/goods/mobilephones/?page=127
Oops. Connection Error occured. Response is: None for url https://torg.mail.ru/review/goods/mobilephones/?page=239
Oops. Connection Error occured. Response is: None for url https://torg.mail.ru/review/goods/mobilephones/?page=

In [318]:
train_df900 = pd.read_csv('data/train_df900.csv', index_col=None, header=0, encoding='cp1251')   

print(train_df900.shape)
train_df900.head()

(17616, 2)


,review,rating
0,"Ужасная камера, отвратительная просто. Звук гр...",2.0
1,Тормозит страшно. Самостоятельно обновляется п...,1.0
2,"\r\nОчень хороший телефон, пользуюсь третий ме...",5.0
3,Айфон спорный аппарат. Начну с недостатков. Ак...,5.0
4,"\r\nПробыл 4 года,батарея стала держать максим...",4.0


## Блок предобработки

### удаление некоторых символов, которые были видны визуально
и/или замена их на пробелы (иногда между знаками препинания и словом пробелы забываются, а лишние пробелы мы потом все равно уберем)

In [6]:
def del_any_symbols(text):
    return (text.replace('\n', ' ').replace('\r', ' ').replace('<br />', ' ').replace('\t', ' ').replace('&quot;', ' ') 
               .replace('.', ' ').replace(',', ' ').replace('!', ' ! ').replace('?', ' ? ').replace(':', ' ').replace(';', ' '))

для **train_df900**:

In [7]:
def del_any_symbols_900(text):
    return (text.replace('\n', ' ').replace('\r', ' ').replace('<br />', ' ').replace('\t', ' ').replace('&quot;', ' ') 
               .replace('!', ' ! ').replace('?', ' ? '))

### удаление пунктуации (замена на пробелы)
(кроме '!' и '?', они могут нести информацию)

In [8]:
# функция для удаления пунктуации
# from string import punctuation
# punctuation: '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
def del_punctuation(text):
    return ''.join(symbol for symbol in text if symbol not in '"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~')

для **train_df900**:

In [9]:
# функция для удаления пунктуации
# from string import punctuation
# punctuation: '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

def del_punctuation_900(text):
    return ''.join(' ' if symbol in '"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~' else symbol for symbol in text)

### лемматизация

In [10]:
import pymorphy2 # с русским языком, в отличие от nltk.stem.WordNetLemmatizer
import re

# функция для лемматизации и удаление лишних пробелов
def lemmatizer(text):
    pymorph = pymorphy2.MorphAnalyzer()
    return' '.join([pymorph.parse(word)[0].normal_form for word in text.split(' ') if word not in ['']])
#     return' '.join([pymorph.parse(word)[0].normal_form for word in text.split(' ') 
#                     if not re.match('([^A-ZА-ЯЁa-zа-яё]+)', word) and word not in ['']])

# PS. normal_form уже предполагает приведение сразу и к lower()

### общий препроцессинг текста

In [11]:
def text_preprocessing(text):
    return lemmatizer(del_punctuation(del_any_symbols(text)))

для **train_df900**:

In [12]:
def text_preprocessing_900(text):
    return lemmatizer(del_punctuation_900(del_any_symbols_900(text)))

In [405]:
%%time
train_df['review_pre'] = train_df['review'].apply(text_preprocessing)

Wall time: 7min 7s


In [412]:
train_df.head()

,rating,review,review_pre
0,2.0,"Ужасная камера, отвратительная просто. Звук гр...",ужасный камера отвратительный просто звук гром...
1,1.0,Тормозит страшно. Самостоятельно обновляется п...,тормозить страшно самостоятельно обновляться п...
2,5.0,"\nОчень хороший телефон, пользуюсь третий меся...",очень хороший телефон пользоваться три месяц
3,5.0,Айфон спорный аппарат. Начну с недостатков. Ак...,айфон спорный аппарат начать с недостаток акку...
4,4.0,"\nПробыл 4 года,батарея стала держать максимум...",пробыть 4 год батарея стать держать максимум т...


In [324]:
%%time
train_df900['review_pre'] = train_df900['review'].apply(text_preprocessing)

Wall time: 35min 21s


In [325]:
train_df900.head()

,review,rating,review_pre
0,"Ужасная камера, отвратительная просто. Звук гр...",2.0,ужасный камера отвратительный просто звук гром...
1,Тормозит страшно. Самостоятельно обновляется п...,1.0,тормозить страшно самостоятельно обновляться п...
2,"\r\nОчень хороший телефон, пользуюсь третий ме...",5.0,очень хороший телефон пользоваться три месяц
3,Айфон спорный аппарат. Начну с недостатков. Ак...,5.0,айфон спорный аппарат начать с недостаток акку...
4,"\r\nПробыл 4 года,батарея стала держать максим...",4.0,пробыть 4 год батарея стать держать максимум т...


In [13]:
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

A Jupyter Widget

In [14]:
%%time
''' Для использования 'progress_apply' вместо 'apply' 
нужно выполнить предварительно однo из следующих:
'''

# from tqdm import tqdm_notebook
# tqdm_notebook().pandas()
# (интерфейс для ноутбука)

'''или'''

# from tqdm import tqdm
# tqdm.pandas()
# (то же, но интерфейс больше для консоли)


'''
см., например, здесь http://qaru.site/questions/201813/progress-indicator-during-pandas-operations-python
'''

test_df['review_pre'] = test_df['review'].progress_apply(text_preprocessing)
test_df['review_pre_900'] = test_df['review'].progress_apply(text_preprocessing_900)

A Jupyter Widget

A Jupyter Widget


Wall time: 27.3 s


In [384]:
test_df.head()

,review,review_pre_900,review_pre
0,"Ужасно слабый аккумулятор, это основной минус ...",ужасно слабый аккумулятор это основной минус э...,ужасно слабый аккумулятор это основной минус э...
1,ценанадежность-неубиваемостьдолго держит батар...,ценанадёжность неубиваемостьдолго держать бата...,ценанадежностьнеубиваемостьдолго держать батар...
2,"подробнее в комментариях\r\nК сожалению, факт ...",дробный в комментарий к сожаление факт поломка...,дробный в комментарий к сожаление факт поломка...
3,я любительница громкой музыки. Тише телефона у...,я любительница громкий музыка тиша телефон у я...,я любительница громкий музыка тиша телефон у я...
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех...",дата выпуск 2011 год ёмкость 1430 mah технолог...,дата выпуск 2011 год ёмкость 1430 mah технолог...


In [338]:
# сохраним
# train_df.to_csv('data/train_df.csv')
# train_df900.to_csv('data/train_df900.csv')
# test_df.to_csv('data/test_df.csv')

## <center>Part 3. Modeling

### <center>Subpart 3.1. Работа с train_df
**(в 3.2. будем работать с train_df900)**

Для считывания делаем так (обратить внимание на **кодировку**, по дефолту дает, как ни странно, ошибку, хотя сохраняли тоже по дефолту):

In [2]:
# загрузим
train_df = pd.read_csv('data/train_df.csv', index_col=0, encoding='cp1251')
test_df = pd.read_csv('data/test_df.csv', index_col=0, encoding='cp1251')

In [3]:
train_df['rating'].value_counts()

5.0    2199
4.0     799
3.0     397
1.0     370
2.0     207
4.5      20
2.5       4
3.5       3
1.5       1
Name: rating, dtype: int64

Видим, что есть несбалансированность

**Бинаризуем** таргет. Для этого сначала отфильтруем (оставим) только крайние оценки.   
(удалим Наны, которые образовались, видимо , при сохранении пустых строк)

In [4]:
train_df_new = train_df[train_df[['rating']].isin([1.0, 1.5, 2.0, 5.0]).values].dropna().copy()
train_df_new.shape

(2773, 3)

In [7]:
train_df_new['y'] = train_df_new['rating'].apply(lambda x: int(x == 5.0))

Итак, все готово для моделирования предсказания тональности отзывов.    

Воспользуемся результатами прошлых работ.

In [6]:
# функция для записи submission  в папку data
def write_submission(filename, pred):
    dict_ = {1:'pos', 0:'neg'}
    pred = list(map(lambda x: dict_[x], pred))
    pd.DataFrame(pred, columns=['y']).to_csv(path_or_buf='data/' + filename, index_label='Id')

In [8]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC

import nltk
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords

In [419]:
%%time

vectorizer = CountVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df_new['review_pre'])
# X_test = vectorizer.transform(test_df['review_pre'])
y_train = train_df_new['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)
# pred = clf.fit(X_train, y_train).predict(X_test)
# write_submission('submiss_LR_C093_bal_4.csv', pred)

acc_score = 0.867120424961
Wall time: 1min 9s


Пробуем **TfidfVectorizer**

In [429]:
%%time
# задаем предобработку и классификатор
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='word', ngram_range=(1, 2))
X_train = vectorizer.fit_transform(train_df_new['review_pre'])
y_train = train_df_new['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)

acc_score = 0.869643725799
Wall time: 1 s


In [420]:
%%time
# задаем предобработку и классификатор
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df_new['review_pre'])
y_train = train_df_new['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)

acc_score = 0.878641060067
Wall time: 30.9 s


Сделаем посылку на **Kaggle**

In [428]:
%%time

vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df_new['review_pre'])
X_test = vectorizer.transform(test_df['review_pre'])
y_train = train_df_new['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)
pred = clf.fit(X_train, y_train).predict(X_test)
write_submission('submiss_LR_bal_pre_new.csv', pred)

acc_score = 0.878641060067
Wall time: 34.6 s


**Kaggle: 0.93999**

### вычислим и сохраним модели для векторизации текста и классификации

In [10]:
%%time
import pickle as pkl
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df_new['review_pre'])
# X_test = vectorizer.transform(test_df['review_pre'])
y_train = train_df_new['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)
clf.fit(X_train, y_train)
# pred = clf.predict(X_test)
# write_submission('submiss_LR_bal_pre_new.csv', pred)
with open('data/mobilereview_model.pkl', 'wb') as model_pkl, \
     open('data/mobilereview_vectorizer.pkl', 'wb') as vectorizer_pkl:
        pkl.dump(vectorizer, vectorizer_pkl), pkl.dump(clf, model_pkl)

acc_score = 0.878822094033
Wall time: 42.5 s


#### Сделаем подбор параметров (точнее - пока только C)

In [434]:
%%time
# задаем предобработку и классификатор
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df_new['review_pre'])
y_train = train_df_new['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

param_grid = {
              'C': np.logspace(-2, 2, 20)              
             }

gridsearch = GridSearchCV(clf, param_grid, scoring='accuracy', n_jobs=-1)
gridsearch.fit(X_train, y_train)

print(gridsearch.best_params_, 'best accuracy =', gridsearch.best_score_)

{'C': 5.4555947811685144} best accuracy = 0.882967230825
Wall time: 3min 40s


Стало немного лучше. Сделаем посылку с **подобранным C**

In [436]:
%%time

vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df_new['review_pre'])
X_test = vectorizer.transform(test_df['review_pre'])
y_train = train_df_new['y']
C = 5.4555947811685144
clf = LogisticRegression(C=C, class_weight='balanced', random_state=17)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)
pred = clf.fit(X_train, y_train).predict(X_test)
write_submission('submiss_LR_bal_C_pre_new.csv', pred)

acc_score = 0.882966551865
Wall time: 39.1 s


**Kaggle: 0.90000**

Пробуем **НАИВНЫЙ БАЙЕС**

In [74]:
from sklearn.naive_bayes import MultinomialNB

In [75]:
%%time
# задаем предобработку и классификатор
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df_new['review_pre'])
y_train = train_df_new['y']
clf = MultinomialNB()

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)

acc_score = 0.791561789308
Wall time: 25.7 s


Не очень.

### ИТОГО:   
текущий результат на **Kaggle - 0.93999**    
возможно на момент проверки он уже улучшен, команда на Leaderboard - **Шерешевский Дмитрий**      
ссылка на **скриншот** с текущим результатом [здесь](https://prnt.sc/ijiy1l)   

**ПРИМЕЧАНИЕ:**    
В процессе работы было также попробовано следующее:
- было спарсено 18000 отзывов. НО: при использовании сильно больше 2000 результат не улучшался, а временные затраты сильно увеличивались
- пробовал делать сбалансированную выборку, искусственно выбирая по 2000 отрицательных и положительных отзывов в трейн - результат ухудшался по сравнению с естественной выборкой и балансе при помощи параметра `balanced`
- пробовал разные варианты бинаризации трейна (например: `1 if 'raiting' >= 4 else 0`), но наилучший результат был при `1 if 'raiting' == 5.0 and 0 if 'raiting' in [1.0, 1.5, 2.0]` и игнорировании остальных отзывов
- регуляризацию `l1`, но результат был сильно хуже
- интересно, что лемматизация результат улучшала не сильно, а иногда и вовсе слегка ухудшала (тем не менее в финальном варианте ее оставил)
- analyzer='word', ngram_range=(1, 2 (или 3)) в vectorizer, но результат был хуже

### <center>Subpart 3.2. Работа с train_df900

Для считывания делаем так (обратить внимание на **кодировку**, по дефолту дает, как ни странно, ошибку, хотя сохраняли тоже по дефолту):

In [339]:
# загрузим
train_df900 = pd.read_csv('data/train_df900.csv', index_col=0, encoding='cp1251')
test_df = pd.read_csv('data/test_df.csv', index_col=0, encoding='cp1251')

In [340]:
train_df900.head()

,review,rating,review_pre
0,"Ужасная камера, отвратительная просто. Звук гр...",2.0,ужасный камера отвратительный просто звук гром...
1,Тормозит страшно. Самостоятельно обновляется п...,1.0,тормозить страшно самостоятельно обновляться п...
2,"\r\r\nОчень хороший телефон, пользуюсь третий ...",5.0,очень хороший телефон пользоваться три месяц
3,Айфон спорный аппарат. Начну с недостатков. Ак...,5.0,айфон спорный аппарат начать с недостаток акку...
4,"\r\r\nПробыл 4 года,батарея стала держать макс...",4.0,пробыть 4 год батарея стать держать максимум т...


In [341]:
test_df.head()

,review,review_pre_900,review_pre
0,"Ужасно слабый аккумулятор, это основной минус ...",ужасно слабый аккумулятор это основной минус э...,ужасно слабый аккумулятор это основной минус э...
1,ценанадежность-неубиваемостьдолго держит батар...,ценанадёжность неубиваемостьдолго держать бата...,ценанадежностьнеубиваемостьдолго держать батар...
2,"подробнее в комментариях\r\nК сожалению, факт ...",дробный в комментарий к сожаление факт поломка...,дробный в комментарий к сожаление факт поломка...
3,я любительница громкой музыки. Тише телефона у...,я любительница громкий музыка тиша телефон у я...,я любительница громкий музыка тиша телефон у я...
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех...",дата выпуск 2011 год ёмкость 1430 mah технолог...,дата выпуск 2011 год ёмкость 1430 mah технолог...


In [344]:
train_df900['rating'].value_counts()

5.0    7805
4.5    3509
4.0    2478
3.0    1215
1.0     933
3.5     888
2.0     367
2.5     325
1.5      92
0.0       4
Name: rating, dtype: int64

Видим, что есть несбалансированность

**Бинаризуем** таргет. Для этого сначала отфильтруем (оставим) только крайние оценки.   
(удалим Наны, которые образовались, видимо , при сохранении пустых строк)

In [431]:
n_pos = train_df900[['rating']].isin([1.0, 1.5, 2.0, 5.0]).values.sum()
n_pos

9197

In [432]:
n_neg = train_df900[['rating']].isin([1.0, 1.5, 2.0]).values.sum()
n_neg

1392

In [427]:
train_df900_new = train_df900[train_df900[['rating']].isin([1.0, 1.5, 2.0, 5.0]).values].dropna().copy()
train_df900_new.shape

(9186, 3)

In [430]:
train_df900_new['y'] = train_df900_new['rating'].apply(lambda x: int(x == 5.0))

сбалансируем выборку через удаление избыточных положительных отзывов

In [433]:
from sklearn.utils import shuffle
#df = shuffle(df)
df_neg = train_df900_new[train_df900_new['y'] == 0]
df_pos = train_df900_new[train_df900_new['y'] == 1].iloc[:len(df_neg)]

train_df900_new_bal = shuffle(pd.concat([df_pos, df_neg])) # соединим и перемешаем

In [445]:
train_df900_new_bal.head(10)

,review,rating,review_pre,y
1167,\r\r\n все нравится\t\t\t\t\t,5.0,весь нравиться,1
1186,"\r\r\n Модель хорошая, особых проблем замечено...",5.0,модель хороший особый проблема заметить не быт...,1
8139,"\r\r\nТелефоном пользовалась почти 5 лет,ни ра...",1.0,телефон пользоваться почти 5 год ни раз не лом...,0
12158,"Одни минусы, плюсов ничтожно мало, как бы вам ...",2.0,один минус плюс ничтожно мало как бы вы не хот...,0
4417,\r\r\nДорогой и мало функциональный с плохой б...,1.0,дорогой и мало функциональный с плохой батарей...,0
444,\r\r\n очень приятно когда за такую цену можно...,5.0,очень приятно когда за такой цена можно приобр...,1
2730,\r\r\n Не съёмная батарея это приговор для тел...,1.0,не съёмный батарея это приговор для телефон !,0
1609,Хотел бы и я оставить свой отзыв и в первую оч...,5.0,хотеть бы и я оставить свой отзыв и в один оче...,1
518,\r\r\n Прекрасное впечатление. Не разочарован....,5.0,прекрасный впечатление не разочаровать советов...,1
2959,Мне даже жаль было ставить 1. Настолько не дор...,1.0,я даже жаль быть ставить 1 настолько не дорабо...,0


Итак, все готово для моделирования предсказания тональности отзывов.    

Воспользуемся результатами прошлых работ.

In [408]:
# функция для записи submission  в папку data
def write_submission(filename, pred):
    dict_ = {1:'pos', 0:'neg'}
    pred = list(map(lambda x: dict_[x], pred))
    pd.DataFrame(pred, columns=['y']).to_csv(path_or_buf='data/' + filename, index_label='Id')

In [2]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC

import nltk
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords

Пробуем **TfidfVectorizer**

Сделаем посылку на **Kaggle**

In [418]:
%%time

vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df900_new['review_pre'])
X_test = vectorizer.transform(test_df['review_pre_900'])
y_train = train_df900_new['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)
pred = clf.fit(X_train, y_train).predict(X_test)
write_submission('submiss_LR_bal_pre_900_new.csv', pred)

acc_score = 0.857817482432
Wall time: 2min 54s


In [426]:
%%time

vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='word', ngram_range=(1, 5))
X_train = vectorizer.fit_transform(train_df900_new['review_pre'])
X_test = vectorizer.transform(test_df['review_pre_900'])
y_train = train_df900_new['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)
pred = clf.fit(X_train, y_train).predict(X_test)
write_submission('submiss_LR_bal_pre_900_newn12.csv', pred)

acc_score = 0.88105406093
Wall time: 18.6 s


**Kaggle: 0.7**

In [437]:
%%time

vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df900_new_bal['review_pre'])
X_test = vectorizer.transform(test_df['review_pre_900'])
y_train = train_df900_new_bal['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)
pred = clf.fit(X_train, y_train).predict(X_test)
write_submission('submiss_LR_bal^2_pre_900_newn310.csv', pred)

acc_score = 0.785788430029
Wall time: 32.7 s


**Kaggle: 0.66**

#### Сделаем подбор параметров (точнее - пока только C)

In [438]:
%%time
# задаем предобработку и классификатор
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df900_new_bal['review_pre'])
y_train = train_df900_new_bal['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

param_grid = {
              'C': np.logspace(-2, 2, 20)              
             }

gridsearch = GridSearchCV(clf, param_grid, scoring='accuracy', n_jobs=-1)
gridsearch.fit(X_train, y_train)

print(gridsearch.best_params_, 'best accuracy =', gridsearch.best_score_)

{'C': 37.92690190732246} best accuracy = 0.803019410496
Wall time: 4min 42s


In [443]:
%%time
# задаем предобработку и классификатор
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df900_new_bal['review_pre'])
y_train = train_df900_new_bal['y']
clf = LogisticRegression(class_weight='balanced', random_state=17)

param_grid = {
              'C': np.logspace(0, 3, 10)              
             }

gridsearch = GridSearchCV(clf, param_grid, scoring='accuracy', n_jobs=-1)
gridsearch.fit(X_train, y_train)

print(gridsearch.best_params_, 'best accuracy =', gridsearch.best_score_)

{'C': 464.15888336127773} best accuracy = 0.802659956866
Wall time: 3min 34s


In [446]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

In [447]:
%%time
# задаем предобработку и классификатор
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df900_new_bal['review_pre'])
y_train = train_df900_new_bal['y']
#clf = BernoulliNB()
clf = MultinomialNB()

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)

acc_score = 0.749490299397
Wall time: 26.5 s


Не очень...

**ВЫВОД по расширенному датасету:**   
на расширенном датасете результаты, почему-то, хуже. Возможные причины этого:
- переобучение
- как вариант - отличаются выборки, то есть отличаются значимости слов

### Пробуем нейронку

In [448]:
from sklearn.neural_network import MLPClassifier

In [ ]:
%%time

vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df900_new_bal['review_pre'])
X_test = vectorizer.transform(test_df['review_pre_900'])
y_train = train_df900_new_bal['y']
clf = MLPClassifier(hidden_layer_sizes=(30), solver='sgd', activation='tanh', 
                    learning_rate_init=0.1, learning_rate='constant', momentum=0.8)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)
pred = clf.fit(X_train, y_train).predict(X_test)
write_submission('submiss_MLP_pre_900_new_bal.csv', pred)

Делаем ту же нейронку, но на первоначальной короткой несбалансированной выборке (на которой ранее результат был 0.94)

In [ ]:
# загрузим... 
train_df = pd.read_csv('data/train_df.csv', index_col=0, encoding='cp1251')
test_df = pd.read_csv('data/test_df.csv', index_col=0, encoding='cp1251')
# и подготовим
train_df_new = train_df[train_df[['rating']].isin([1.0, 1.5, 2.0, 5.0]).values].dropna().copy()
train_df_new['y'] = train_df_new['rating'].apply(lambda x: int(x == 5.0))

In [ ]:
%%time

vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='char', ngram_range=(3, 10))
X_train = vectorizer.fit_transform(train_df_new['review_pre'])
X_test = vectorizer.transform(test_df['review_pre'])
y_train = train_df_new['y']
clf = MLPClassifier(hidden_layer_sizes=(30), solver='sgd', activation='tanh', 
                    learning_rate_init=0.1, learning_rate='constant', momentum=0.8)

acc_score = cross_val_score(clf, X_train, y_train, scoring='accuracy').mean()
print('acc_score =', acc_score)
pred = clf.fit(X_train, y_train).predict(X_test)
write_submission('submiss_MLP_pre_new.csv', pred)

Нейронка считается очень долго (я за сутки так и не дождался), запустил на **azure.com**